<a href="https://colab.research.google.com/github/Elaineclarindo/PowerBI-DataScience/blob/master/scrapingfutbol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introdução

[](http://)![understat.JPG](http://sergilehkyi.com/wp-content/uploads/2019/06/understat.jpg)

Neste bloco de notas, descreverei o processo de coleta de dados do portal da web [understat.com] (https://understat.com) que contém muitas informações estatísticas sobre todos os jogos nas 5 principais ligas europeias de futebol.

Na página inicial de [understat.com] (https://understat.com):

* Metas esperadas (xG) é a nova métrica revolucionária do futebol, que permite avaliar o desempenho do time e do jogador.

* Em um jogo de baixa pontuação como o futebol, o resultado final da partida não fornece uma imagem clara do desempenho.

* É por isso que cada vez mais os analíticos esportivos recorrem a modelos avançados como o xG, que é uma medida estatística da qualidade das chances criadas e concedidas.

* Nosso objetivo era criar o método mais preciso para avaliação da qualidade do tiro.

* Para este caso, treinamos algoritmos de predição de rede neural com o grande conjunto de dados (> 100.000 fotos, mais de 10 parâmetros para cada).

* Neste site, você encontrará nossas estatísticas xG detalhadas para as principais ligas europeias.

No momento eles não possuem apenas a métrica xG, mas muito mais, o que torna este site perfeito para obter dados estatísticos sobre jogos de futebol.


At this moment they have not only xG metric, but much more, that makes this site perfect for scraping statistical data about football games.




Começamos importando as bibliotecas que serão utilizadas neste projeto:
* numpy - pacote fundamental para computação científica com Python
* pandas - biblioteca que fornece estruturas de dados e ferramentas de análise de dados fáceis de usar e de alto desempenho
* solicitações - é a única biblioteca HTTP não-GMO para Python, segura para consumo humano. (adoro esta linha dos documentos oficiais: D)
* BeautifulSoup - uma biblioteca Python para extrair dados de arquivos HTML e XML.

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import requests
from bs4 import BeautifulSoup

## Pesquisa de site e estrutura de dados

Na página inicial podemos notar que o site possui dados de 6 Ligas Européias:

![leagues.jpg](http://sergilehkyi.com/wp-content/uploads/2019/06/leagues.jpg)

*   La Liga
*   EPL
*   BundesLiga
*   Serie A
*   Ligue 1
*   RFPL

E também vemos que os dados coletados são a partir da temporada 2014/2015. Outra noção que fazemos é a estrutura da URL. Isto é '`https://understat.com/league'` + '`/name_of_the_league`' + '`/year_start_of_the_season`'

![seasons.jpg](http://sergilehkyi.com/wp-content/uploads/2019/06/seasons.jpg)

Portanto, criamos variáveis globais com esses dados para poder selecionar qualquer um deles.

In [4]:
# create urls for all seasons of all leagues
base_url = 'https://understat.com/league'
leagues = ['La_liga', 'EPL', 'Bundesliga', 'Serie_A', 'Ligue_1', 'RFPL']
seasons = ['2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']

O próximo passo é entender onde os dados estão localizados na página da web. Para isso, abrimos as Ferramentas do desenvolvedor no Chrome, vá para a aba "Rede", encontre o arquivo com os dados (neste caso, 2018) e verifique a aba "Resposta". Isso é o que obteremos após executar * requests.get (URL) *

![requests_response_1.jpg](http://sergilehkyi.com/wp-content/uploads/2019/06/requests_response_1.jpg)

Depois de passar pelo conteúdo da página da web, descobrimos que os dados são armazenados na tag "script" e são codificados em JSON. Portanto, precisaremos encontrar essa tag, obter JSON a partir dela e convertê-la em uma estrutura de dados legível em Python.

![requests_response_2.jpg](http://sergilehkyi.com/wp-content/uploads/2019/06/requests_response_2.jpg)

In [8]:
# Starting with latest data for Spanish league, because I'm a Barcelona fan
url = base_url+'/'+leagues[0]+'/'+seasons[4]
res = requests.get(url)
soup = BeautifulSoup(res.content, "lxml")

# Based on the structure of the webpage, I found that data is in the JSON variable, under <script> tags
scripts = soup.find_all('script')

# Check our <script> tags
for el in scripts:
  print('*'*50)
  print(el.text)

**************************************************

			var THEME = localStorage.getItem("theme") || 'DARK';
			document.body.className = "theme-" + THEME.toLowerCase();
		
**************************************************

	var datesData 	= JSON.parse('\x5B\x7B\x22id\x22\x3A\x229957\x22,\x22isResult\x22\x3Atrue,\x22h\x22\x3A\x7B\x22id\x22\x3A\x22223\x22,\x22title\x22\x3A\x22Girona\x22,\x22short_title\x22\x3A\x22GIR\x22\x7D,\x22a\x22\x3A\x7B\x22id\x22\x3A\x22231\x22,\x22title\x22\x3A\x22Real\x20Valladolid\x22,\x22short_title\x22\x3A\x22VAL\x22\x7D,\x22goals\x22\x3A\x7B\x22h\x22\x3A\x220\x22,\x22a\x22\x3A\x220\x22\x7D,\x22xG\x22\x3A\x7B\x22h\x22\x3A\x220.626716\x22,\x22a\x22\x3A\x220.030934\x22\x7D,\x22datetime\x22\x3A\x222018\x2D08\x2D17\x2021\x3A15\x3A00\x22,\x22forecast\x22\x3A\x7B\x22w\x22\x3A\x220.4708\x22,\x22d\x22\x3A\x220.5147\x22,\x22l\x22\x3A\x220.0145\x22\x7D\x7D,\x7B\x22id\x22\x3A\x229958\x22,\x22isResult\x22\x3Atrue,\x22h\x22\x3A\x7B\x22id\x22\x3A\x22153\x22,\x22title\x22\x

### Trabalhando com  JSON

Descobrimos que os dados que nos interessam são armazenados na variável teamsData, depois de criar uma sopa de tags html ela se torna apenas uma string, então encontramos esse texto e extraímos JSON dele.

In [10]:
import json

string_with_json_obj = ''

# Find data for teams
for el in scripts:
    if 'teamsData' in el.text:
      string_with_json_obj = el.text.strip()
      
# print(string_with_json_obj)

# strip unnecessary symbols and get only JSON data
ind_start = string_with_json_obj.index("('")+2
ind_end = string_with_json_obj.index("')")
json_data = string_with_json_obj[ind_start:ind_end]

json_data = json_data.encode('utf8').decode('unicode_escape')

Depois de obter nosso JSON e limpá-lo, podemos convertê-lo em um dicionário Python e verificar sua aparência (descomente a instrução print para fazer isso).

### Understanding data with Python

In [11]:
# convert JSON data into Python dictionary
data = json.loads(json_data)
print(data.keys())
print('='*50)
print(data['138'].keys())
print('='*50)
print(data['138']['id'])
print('='*50)
print(data['138']['title'])
print('='*50)
print(data['138']['history'][0])

# Print pretty JSON data to check out what we have there
# s = json.dumps(data, indent=4, sort_keys=True)
# print(s)

dict_keys(['138', '140', '141', '142', '143', '145', '146', '147', '148', '150', '151', '152', '153', '154', '156', '158', '159', '223', '231', '232'])
dict_keys(['id', 'title', 'history'])
138
Sevilla
{'h_a': 'a', 'xG': 3.34997, 'xGA': 2.39239, 'npxG': 3.34997, 'npxGA': 1.64976, 'ppda': {'att': 252, 'def': 19}, 'ppda_allowed': {'att': 263, 'def': 20}, 'deep': 12, 'deep_allowed': 5, 'scored': 4, 'missed': 1, 'xpts': 1.9829999999999999, 'result': 'w', 'date': '2018-08-19 21:15:00', 'wins': 1, 'draws': 0, 'loses': 0, 'pts': 3, 'npxGD': 1.70021}


Se você quiser verificar a aparência de todos os <code> dados </code>, apenas descomente as respectivas linhas. (é comentado para economizar espaço na tela e não sobrecarrega a visualização do notebook).

Quando começamos a pesquisar os <code> dados </code> entendemos que se trata de um dicionário de dicionários de 3 chaves: * `id` *, *` title` * e * `history` *. A primeira camada do dicionário também usa ids como chaves.

Também a partir disso entendemos que * `histórico` * contém dados sobre cada jogo que a equipe jogou em sua própria liga (jogos da Copa da Liga ou da Liga dos Campeões não estão incluídos).

Podemos reunir nomes de equipes se examinarmos o dicionário da primeira camada.

In [12]:
# Get teams and their relevant ids and put them into separate dictionary
teams = {}
for id in data.keys():
  teams[id] = data[id]['title']

O * `histórico` * é a matriz de dicionários onde as chaves são nomes de métricas (leia nomes de colunas) e valores são valores, apesar de quão tautológico é isso: D.

Entendemos que os nomes das colunas se repetem continuamente, por isso os adicionamos a uma lista separada. Também verificando a aparência dos valores de amostra.

In [13]:
# EDA to get a feeling of how the JSON is structured
# Column names are all the same, so we just use first element
columns = []
# Check the sample of values per each column
values = []
for id in data.keys():
  columns = list(data[id]['history'][0].keys())
  values = list(data[id]['history'][0].values())
  break

print(columns)
print(values)

['h_a', 'xG', 'xGA', 'npxG', 'npxGA', 'ppda', 'ppda_allowed', 'deep', 'deep_allowed', 'scored', 'missed', 'xpts', 'result', 'date', 'wins', 'draws', 'loses', 'pts', 'npxGD']
['a', 3.34997, 2.39239, 3.34997, 1.64976, {'att': 252, 'def': 19}, {'att': 263, 'def': 20}, 12, 5, 4, 1, 1.9829999999999999, 'w', '2018-08-19 21:15:00', 1, 0, 0, 3, 1.70021]


Constatou que o Sevilla tem o id = 138, obtendo assim todos os dados para que este time possa reproduzir os mesmos passos para todos os times da liga.

In [14]:
sevilla_data = []
for row in data['138']['history']:
  sevilla_data.append(list(row.values()))
  
df = pd.DataFrame(sevilla_data, columns=columns)
df.head(2)

,h_a,xG,xGA,npxG,npxGA,ppda,ppda_allowed,deep,deep_allowed,scored,missed,xpts,result,date,wins,draws,loses,pts,npxGD
0,a,3.34997,2.392390,3.34997,1.649760,"{'att': 252, 'def': 19}","{'att': 263, 'def': 20}",12,5,4,1,1.9830,w,2018-08-19 21:15:00,1,0,0,3,1.700210
1,h,1.97161,0.671429,1.97161,0.671429,"{'att': 262, 'def': 16}","{'att': 237, 'def': 26}",11,3,0,0,2.3331,d,2018-08-26 21:15:00,0,1,0,1,1.300181


Wualya! Temos os dados de todos os jogos do Sevilla na temporada 2018-2019 dentro da La Liga!

Agora queremos fazer isso para todas as equipes espanholas. Vamos meus alunos do BI

In [16]:
# Getting data for all teams
dataframes = {}
for id, team in teams.items():
  teams_data = []
  for row in data[id]['history']:
    teams_data.append(list(row.values()))
    
  df = pd.DataFrame(teams_data, columns=columns)
  dataframes[team] = df
  print('Added data for {}.'.format(team))
  

Added data for Sevilla.
Added data for Real Sociedad.
Added data for Espanyol.
Added data for Getafe.
Added data for Atletico Madrid.
Added data for Rayo Vallecano.
Added data for Valencia.
Added data for Athletic Club.
Added data for Barcelona.
Added data for Real Madrid.
Added data for Levante.
Added data for Celta Vigo.
Added data for Real Betis.
Added data for Villarreal.
Added data for Eibar.
Added data for Alaves.
Added data for Leganes.
Added data for Girona.
Added data for Real Valladolid.
Added data for SD Huesca.


Agora temos um dicionário de DataFrames onde key é o nome do time e value é o DataFrame com todos os jogos desse time.

In [17]:
# Sample check of our newly created DataFrame
dataframes['Barcelona'].head(2)

,h_a,xG,xGA,npxG,npxGA,ppda,ppda_allowed,deep,deep_allowed,scored,missed,xpts,result,date,wins,draws,loses,pts,npxGD
0,h,3.26753,0.248353,3.26753,0.248353,"{'att': 118, 'def': 17}","{'att': 407, 'def': 13}",20,0,3,0,2.9009,w,2018-08-18 23:15:00,1,0,0,3,3.019177
1,a,1.20392,0.510742,1.20392,0.510742,"{'att': 163, 'def': 16}","{'att': 316, 'def': 15}",15,4,1,0,1.9865,w,2018-08-25 23:15:00,1,0,0,3,0.693178


### Manipulations to make data as in the original source

Podemos notar que aqui métricas como PPDA e OPPDA (ppda e ppda_allowed) são representadas como montantes totais de ações de ataque / defesa, mas na tabela original é mostrado como coeficiente. Vamos consertar isso!

In [18]:
for team, df in dataframes.items():
  dataframes[team]['ppda_coef'] = dataframes[team]['ppda'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)
  dataframes[team]['oppda_coef'] = dataframes[team]['ppda_allowed'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)
  
# And check how our new dataframes look based on Sevilla dataframe
dataframes['Sevilla'].head(2)

,h_a,xG,xGA,npxG,npxGA,ppda,ppda_allowed,deep,deep_allowed,scored,missed,xpts,result,date,wins,draws,loses,pts,npxGD,ppda_coef,oppda_coef
0,a,3.34997,2.392390,3.34997,1.649760,"{'att': 252, 'def': 19}","{'att': 263, 'def': 20}",12,5,4,1,1.9830,w,2018-08-19 21:15:00,1,0,0,3,1.700210,13.263158,13.150000
1,h,1.97161,0.671429,1.97161,0.671429,"{'att': 262, 'def': 16}","{'att': 237, 'def': 26}",11,3,0,0,2.3331,d,2018-08-26 21:15:00,0,1,0,1,1.300181,16.375000,9.115385


Agora temos todos os nossos números, mas para cada jogo. O que precisamos é dos totais para a equipe. Vamos descobrir as colunas que temos que resumir. Para isso, voltamos à tabela original em [understat.com] (https://understat.com/league/La_liga/2018) e descobrimos que todas as métricas devem ser somadas e apenas PPDA e OPPDA são meios no final.

In [19]:
cols_to_sum = ['xG', 'xGA', 'npxG', 'npxGA', 'deep', 'deep_allowed', 'scored', 'missed', 'xpts', 'wins', 'draws', 'loses', 'pts', 'npxGD']
cols_to_mean = ['ppda_coef', 'oppda_coef']

Estamos prontos para calcular nossos totais e médias. Para isso, percorremos o dicionário de dataframes e chamamos os métodos .sum () e .mean () DataFrame que retornam Series, é por isso que adicionamos .transpose () a essas chamadas. Colocamos esses novos DataFrames em uma lista e depois disso os concatamos em um novo DataFrame `full_stat`

In [20]:
frames = []
for team, df in dataframes.items():
  sum_data = pd.DataFrame(df[cols_to_sum].sum()).transpose()
  mean_data = pd.DataFrame(df[cols_to_mean].mean()).transpose()
  final_df = sum_data.join(mean_data)
  final_df['team'] = team
  final_df['matches'] = len(df)
  frames.append(final_df)
  
full_stat = pd.concat(frames)

Em seguida, reordenamos as colunas para melhor legibilidade, classificamos as linhas com base em pontos, redefinimos o índice e adicionamos a 'posição' da coluna.

In [21]:
full_stat = full_stat[['team', 'matches', 'wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'xG', 'npxG', 'xGA', 'npxGA', 'npxGD', 'ppda_coef', 'oppda_coef', 'deep', 'deep_allowed', 'xpts']]
full_stat.sort_values('pts', ascending=False, inplace=True)
full_stat.reset_index(inplace=True, drop=True)
full_stat['position'] = range(1,len(full_stat)+1)

Também na tabela original temos valores de diferenças entre as métricas esperadas e reais. Vamos adicionar esses também.

In [22]:
full_stat['xG_diff'] = full_stat['xG'] - full_stat['scored']
full_stat['xGA_diff'] = full_stat['xGA'] - full_stat['missed']
full_stat['xpts_diff'] = full_stat['xpts'] - full_stat['pts']

Converter flutuações em inteiros quando apropriado

In [23]:
cols_to_int = ['wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'deep', 'deep_allowed']
full_stat[cols_to_int] = full_stat[cols_to_int].astype(int)

Produção de enfeites e visão final de um DataFrame

In [24]:
col_order = ['position','team', 'matches', 'wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'xG', 'xG_diff', 'npxG', 'xGA', 'xGA_diff', 'npxGA', 'npxGD', 'ppda_coef', 'oppda_coef', 'deep', 'deep_allowed', 'xpts', 'xpts_diff']
full_stat = full_stat[col_order]
pd.options.display.float_format = '{:,.2f}'.format
full_stat.head(10)

,position,team,matches,wins,draws,loses,scored,missed,pts,xG,xG_diff,npxG,xGA,xGA_diff,npxGA,npxGD,ppda_coef,oppda_coef,deep,deep_allowed,xpts,xpts_diff
0,1,Barcelona,38,26,9,3,90,36,87,83.28,-6.72,76.58,44.93,8.93,43.44,33.14,9.02,16.40,417,171,73.96,-13.04
1,2,Atletico Madrid,38,22,10,6,55,29,76,51.87,-3.13,48.73,41.43,12.43,37.72,11.01,11.07,11.10,252,190,59.43,-16.57
2,3,Real Madrid,38,21,5,12,63,46,68,68.65,5.65,61.97,48.68,2.68,42.73,19.24,8.90,14.78,341,168,64.77,-3.23
3,4,Valencia,38,15,16,7,51,35,61,61.88,10.88,56.57,42.85,7.85,36.91,19.66,12.96,9.47,278,215,65.16,4.16
4,5,Sevilla,38,17,8,13,62,47,59,69.16,7.16,64.54,46.71,-0.29,41.51,23.03,10.65,10.02,321,211,65.08,6.08
5,6,Getafe,38,15,14,9,48,35,59,47.03,-0.97,42.58,44.23,9.23,39.02,3.56,8.77,5.70,186,196,53.19,-5.81
6,7,Espanyol,38,14,11,13,48,50,53,50.16,2.16,47.18,54.62,4.62,48.55,-1.36,9.86,9.82,241,241,50.09,-2.91
7,8,Athletic Club,38,13,14,11,41,45,53,44.44,3.44,38.92,47.16,2.16,43.44,-4.53,8.30,11.30,221,185,50.01,-2.99
8,9,Real Sociedad,38,13,11,14,45,46,50,47.99,2.99,40.55,48.09,2.09,45.68,-5.13,9.94,9.49,194,208,51.13,1.13
9,10,Alaves,38,13,11,14,39,50,50,40.87,1.87,38.64,54.53,4.53,50.07,-11.43,11.23,7.10,129,270,44.02,-5.98


Original table

![full_table.JPG](http://sergilehkyi.com/wp-content/uploads/2019/06/full_table.jpg)

## Scraping data para todos os times em um session

Testar o fluxo antes de entrar totalmente no processo

In [25]:
season_data = dict()
season_data[seasons[4]] = full_stat
print(season_data)
full_data = dict()
full_data[leagues[0]] = season_data
print(full_data)

{'2018':     position             team  matches  ...  deep_allowed  xpts  xpts_diff
0          1        Barcelona       38  ...           171 73.96     -13.04
1          2  Atletico Madrid       38  ...           190 59.43     -16.57
2          3      Real Madrid       38  ...           168 64.77      -3.23
3          4         Valencia       38  ...           215 65.16       4.16
4          5          Sevilla       38  ...           211 65.08       6.08
5          6           Getafe       38  ...           196 53.19      -5.81
6          7         Espanyol       38  ...           241 50.09      -2.91
7          8    Athletic Club       38  ...           185 50.01      -2.99
8          9    Real Sociedad       38  ...           208 51.13       1.13
9         10           Alaves       38  ...           270 44.02      -5.98
10        11       Real Betis       38  ...           220 52.39       2.39
11        12            Eibar       38  ...           177 56.75       9.75
12        13    



Colocar todo o código anterior em loops para obter todos os dados.

In [26]:
full_data = dict()
for league in leagues:
  
  season_data = dict()
  for season in seasons:    
    url = base_url+'/'+league+'/'+season
    res = requests.get(url)
    soup = BeautifulSoup(res.content, "lxml")

    # Based on the structure of the webpage, I found that data is in the JSON variable, under <script> tags
    scripts = soup.find_all('script')
    
    string_with_json_obj = ''

    # Find data for teams
    for el in scripts:
        if 'teamsData' in el.text:
          string_with_json_obj = el.text.strip()

    # print(string_with_json_obj)

    # strip unnecessary symbols and get only JSON data
    ind_start = string_with_json_obj.index("('")+2
    ind_end = string_with_json_obj.index("')")
    json_data = string_with_json_obj[ind_start:ind_end]
    json_data = json_data.encode('utf8').decode('unicode_escape')
    
    
    # convert JSON data into Python dictionary
    data = json.loads(json_data)
    
    # Get teams and their relevant ids and put them into separate dictionary
    teams = {}
    for id in data.keys():
      teams[id] = data[id]['title']
      
    # EDA to get a feeling of how the JSON is structured
    # Column names are all the same, so we just use first element
    columns = []
    # Check the sample of values per each column
    values = []
    for id in data.keys():
      columns = list(data[id]['history'][0].keys())
      values = list(data[id]['history'][0].values())
      break
      
    # Getting data for all teams
    dataframes = {}
    for id, team in teams.items():
      teams_data = []
      for row in data[id]['history']:
        teams_data.append(list(row.values()))

      df = pd.DataFrame(teams_data, columns=columns)
      dataframes[team] = df
      # print('Added data for {}.'.format(team))
      
    
    for team, df in dataframes.items():
      dataframes[team]['ppda_coef'] = dataframes[team]['ppda'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)
      dataframes[team]['oppda_coef'] = dataframes[team]['ppda_allowed'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)
      
    cols_to_sum = ['xG', 'xGA', 'npxG', 'npxGA', 'deep', 'deep_allowed', 'scored', 'missed', 'xpts', 'wins', 'draws', 'loses', 'pts', 'npxGD']
    cols_to_mean = ['ppda_coef', 'oppda_coef']
    
    frames = []
    for team, df in dataframes.items():
      sum_data = pd.DataFrame(df[cols_to_sum].sum()).transpose()
      mean_data = pd.DataFrame(df[cols_to_mean].mean()).transpose()
      final_df = sum_data.join(mean_data)
      final_df['team'] = team
      final_df['matches'] = len(df)
      frames.append(final_df)

    full_stat = pd.concat(frames)
    
    full_stat = full_stat[['team', 'matches', 'wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'xG', 'npxG', 'xGA', 'npxGA', 'npxGD', 'ppda_coef', 'oppda_coef', 'deep', 'deep_allowed', 'xpts']]
    full_stat.sort_values('pts', ascending=False, inplace=True)
    full_stat.reset_index(inplace=True, drop=True)
    full_stat['position'] = range(1,len(full_stat)+1)
    
    full_stat['xG_diff'] = full_stat['xG'] - full_stat['scored']
    full_stat['xGA_diff'] = full_stat['xGA'] - full_stat['missed']
    full_stat['xpts_diff'] = full_stat['xpts'] - full_stat['pts']
    
    cols_to_int = ['wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'deep', 'deep_allowed']
    full_stat[cols_to_int] = full_stat[cols_to_int].astype(int)
    
    col_order = ['position', 'team', 'matches', 'wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'xG', 'xG_diff', 'npxG', 'xGA', 'xGA_diff', 'npxGA', 'npxGD', 'ppda_coef', 'oppda_coef', 'deep', 'deep_allowed', 'xpts', 'xpts_diff']
    full_stat = full_stat[col_order]
    full_stat = full_stat.set_index('position')
    # print(full_stat.head(20))
    
    season_data[season] = full_stat
  
  df_season = pd.concat(season_data)
  full_data[league] = df_season
  
data = pd.concat(full_data)
data.head()
  

team  matches  ...  xpts  xpts_diff
             position                            ...                 
La_liga 2014 1               Barcelona       38  ... 94.08       0.08
             2             Real Madrid       38  ... 81.75     -10.25
             3         Atletico Madrid       38  ... 73.14      -4.86
             4                Valencia       38  ... 63.71     -13.29
             5                 Sevilla       38  ... 67.39      -8.61

[5 rows x 21 columns]

## Exporting data to CSV file

In [27]:
data.to_csv('understat.com.csv')